In [1]:
!pip -q install torch_geometric rectools
!pip -q install comet_ml
!pip -q install python-dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 50.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.0/208.0 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 114.3 MB/s eta 0:00:0000:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 1.42.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 727.1/727.1 kB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 55.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 85.3 MB/s eta 0:00:00:00:01


In [2]:
import comet_ml
from comet_ml import Experiment
from comet_ml.integration.pytorch import log_model

from dotenv import load_dotenv
import os

In [ ]:
load_dotenv(".env")

True

In [4]:
experiment = Experiment(
  api_key=os.getenv('API_KEY'),
  project_name="gnn-recommender",
  workspace="annanet",
  log_code=True
)

experiment.set_name('diffGATGraphModel-beauty')
experiment.add_tags(['beauty', 'leave-n-out'])

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/annanet/gnn-recommender/88062f9b711a4b7a8f06c94568841b6d

COMET INFO: Couldn't find a Git repository in '/kaggle/working' nor in any parent directory. Set `COMET_GIT_DIRECTORY` if your Git Repository is elsewhere.


In [5]:
hyperparameters = {
    'seed': 42,
    'types_of_feedback': ["explicit_positive", "expliсit_negative",
                          "implicit_positive", "implicit_negative"],
    'train_edge_type': ('item','to_feedback_explicit_positive','explicit_positive'),
    'train_num_epochs': 100,
    'train_lr': 8e-5,
    'train_batch_size': 4096,
    'train_print_every': 10,  
    'train_test_every': 10,
    'test_topk': 10,
    'test_batch_size': 2048
}

In [6]:
import os
os.listdir('/kaggle/input/data/leave-n-out/beauty')

['train.csv', 'test.csv']

In [7]:
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import HeteroData
from torch_geometric.nn import HeteroConv, SAGEConv, GATConv

from sklearn.preprocessing import LabelEncoder

from rectools import Columns
from rectools.metrics import MAP, Precision, Recall, NDCG, calc_metrics

import gc
import random

In [8]:
SEED = hyperparameters['seed']
torch.manual_seed(SEED)
random.seed(SEED)
np.random.seed(SEED)

In [9]:
rootpath = '/kaggle/input/data/leave-n-out/beauty/'
train = pd.read_csv(
    rootpath+'train.csv'
)
train['date'] = pd.to_datetime(train['unix_time'])
print(train.head())

                 user_id     item_id  rating  \
0  A00414041RD0BXM6WK0GX  B007IY97U0     3.0   
1  A00414041RD0BXM6WK0GX  B00870XLDS     2.0   
2  A00414041RD0BXM6WK0GX  B008MIRO88     1.0   
3  A00414041RD0BXM6WK0GX  B00BQYYMN0     3.0   
4  A00414041RD0BXM6WK0GX  B00GRTQBTM     5.0   

                                         review_text   unix_time       date  
0  Good quality wig, but the blonde is much more ...  2014-07-14 2014-07-14  
1  Very thin and not as long as the photos :( Aft...  2014-07-14 2014-07-14  
2  Very thin and not as long as the photos :( Aft...  2014-07-14 2014-07-14  
3  This is a great quality wig, however it is a m...  2014-07-14 2014-07-14  
4  This is my absolute favorite wig! I have purch...  2014-07-14 2014-07-14  


In [10]:
explicit_positive = train[(train["rating"] == 5)].index
explisit_negative = train[(train["rating"] <= 2)].index

explicit_combined_feedback = explicit_positive.union(explisit_negative)
print('Количество explicit позитивного фидбека', explicit_positive.shape[0])
print('Количество explicit негативного фидбека', explisit_negative.shape[0])

Количество explicit позитивного фидбека 90800
Количество explicit негативного фидбека 17504


In [11]:
implicit_positive = train[(train["rating"] == 4)].index
implicit_negative = train[(train["rating"] == 3)].index

implicit_combined_feedback = implicit_positive.union(implicit_negative)
print('Количество implicit позитивного фидбека', implicit_positive.shape[0])
print('Количество implicit негативного фидбека', implicit_negative.shape[0])

Количество implicit позитивного фидбека 30668
Количество implicit негативного фидбека 17110


In [12]:
train.loc[:, "target"] = ""
train.loc[explicit_positive, "target"] = "explicit_positive"
train.loc[explisit_negative, "target"] = "expliсit_negative"
train.loc[implicit_positive, "target"] = "implicit_positive"
train.loc[implicit_negative, "target"] = "implicit_negative"

train = train[['user_id','item_id','target','date']]
train.head()

,user_id,item_id,target,date
0,A00414041RD0BXM6WK0GX,B007IY97U0,implicit_negative,2014-07-14
1,A00414041RD0BXM6WK0GX,B00870XLDS,expliсit_negative,2014-07-14
2,A00414041RD0BXM6WK0GX,B008MIRO88,expliсit_negative,2014-07-14
3,A00414041RD0BXM6WK0GX,B00BQYYMN0,implicit_negative,2014-07-14
4,A00414041RD0BXM6WK0GX,B00GRTQBTM,explicit_positive,2014-07-14


In [13]:
train = train.sort_values(by=["user_id", "date"]).reset_index(drop=True)
train.columns = ['user_id', 'item_id', 'target', 'date']

In [14]:
test = pd.read_csv(
    rootpath+'test.csv'
)
test['date'] = pd.to_datetime(test['unix_time'])
print(test.head())

                 user_id     item_id  rating  \
0  A02155413BVL8D0G7X6DN  B0089JVEPO     5.0   
1  A02155413BVL8D0G7X6DN  B001G2LWDK     5.0   
2  A02155413BVL8D0G7X6DN  B005Z41P28     5.0   
3  A02155413BVL8D0G7X6DN  B0055MYJ0U     5.0   
4  A02155413BVL8D0G7X6DN  B00117CH5M     3.0   

                                         review_text   unix_time       date  
0  leaves my skin clean and smooth. it is creamy ...  2012-10-25 2012-10-25  
1  Works great, smells good, there is a result. I...  2012-12-06 2012-12-06  
2  it works for my hair. smells like almond. made...  2013-01-17 2013-01-17  
3  got this in the mail from China today! holds m...  2013-04-22 2013-04-22  
4  if you like strong smell of honeysuckles and h...  2013-05-01 2013-05-01  


In [15]:
test = test[['user_id','item_id', 'date']]
test.columns = ['user_id', 'item_id', 'date']
test.head()

,user_id,item_id,date
0,A02155413BVL8D0G7X6DN,B0089JVEPO,2012-10-25
1,A02155413BVL8D0G7X6DN,B001G2LWDK,2012-12-06
2,A02155413BVL8D0G7X6DN,B005Z41P28,2013-01-17
3,A02155413BVL8D0G7X6DN,B0055MYJ0U,2013-04-22
4,A02155413BVL8D0G7X6DN,B00117CH5M,2013-05-01


# MVP model v2

In [16]:
test = test[(test.user_id.isin(train.user_id)) & (test.item_id.isin(train.item_id))].copy()
test.shape

(42378, 3)

In [17]:
# 2. Преобразование данных - для куарека не особо нужно, но для других - напоминалка
# делаем всегда! чтобы не сломать ничего дальше и чтобы все индексы были от 0 до N без пропусков
user_encoder = LabelEncoder()
video_encoder = LabelEncoder()

train.loc[:, 'user_id'] = user_encoder.fit_transform(train['user_id'])
train.loc[:, 'item_id'] = video_encoder.fit_transform(train['item_id'])

test.loc[:, 'user_id'] = user_encoder.transform(test['user_id'])
test.loc[:, 'item_id'] = video_encoder.transform(test['item_id'])

train['user_id'] = train['user_id'].astype(int)
train['item_id'] = train['item_id'].astype(int)
test['user_id'] = test['user_id'].astype(int)
test['item_id'] = test['item_id'].astype(int)

In [18]:
# т.е. сразу знаем количество и в каких пределах изменяется user_id и video_id
num_videos = train['item_id'].nunique()
num_users = train['user_id'].nunique()

print('Количество уникальных item_id', num_videos)
print('Количество уникальных user_id', num_users)

Количество уникальных item_id 12095
Количество уникальных user_id 22363


In [19]:
def prepare_hetero_data(df) -> HeteroData:
    data = HeteroData()
    users = df['user_id'].unique()
    num_users = len(users)
    num_items = df['item_id'].max() + 1
    feedback_types = df['target'].unique().tolist()

    # узлы
    data['user'].node_id = torch.arange(num_users)
    data['item'].node_id = torch.arange(num_items)
    for ft in feedback_types:
        data[ft].node_id = torch.arange(num_users)

    # ребра item<->feedback->user
    for ft in feedback_types:
        mask = df['target'] == ft
        items = torch.LongTensor(df.loc[mask, 'item_id'].values)
        users_idx = torch.LongTensor(df.loc[mask, 'user_id'].values)
        # item -> fb
        data['item', f'to_feedback_{ft}', ft].edge_index = torch.stack([items, users_idx], dim=0)
        # fb -> item
        data[ft, f'feedback_to_item_{ft}', 'item'].edge_index = torch.stack([users_idx, items], dim=0)
        # fb -> user (1:1)
        idx = torch.arange(num_users)
        data[ft, f'to_user_{ft}', 'user'].edge_index = torch.stack([idx, idx], dim=0)

    return data

In [20]:
class SignedHeteroGNN(nn.Module):
    def __init__(self,
                 num_users: int,
                 num_items: int,
                 feedback_types: list,
                 emb_dim: int = 32,
                 hidden_dim: int = 16,
                 heads: int = 2,
                 dropout: float = 0.2):
        super().__init__()
        self.feedback_types = feedback_types
        self.pos_types = ['implicit_positive', 'explicit_positive']
        self.neg_types = ['implicit_negative', 'expliсit_negative']

        self.user_emb = nn.Embedding(num_users, emb_dim)
        self.item_emb = nn.Embedding(num_items, emb_dim)
        self.fb_emb   = nn.ModuleDict({
            ft: nn.Embedding(num_users, emb_dim) for ft in feedback_types
        })

        conv1, conv2 = {}, {}
        for ft in feedback_types:
            # ft -> user
            conv1[(ft, f'to_user_{ft}', 'user')] = GATConv(emb_dim, hidden_dim,
                                                      heads=heads,
                                                      add_self_loops=False)
            conv2[(ft, f'to_user_{ft}', 'user')] = GATConv(hidden_dim*heads, emb_dim,
                                                      heads=1,
                                                      add_self_loops=False)
            # ft -> item
            conv1[(ft, f'feedback_to_item_{ft}', 'item')] = GATConv(emb_dim, hidden_dim,
                                                       heads=heads,
                                                       add_self_loops=False)
            conv2[(ft, f'feedback_to_item_{ft}', 'item')] = GATConv(hidden_dim*heads, emb_dim,
                                                       heads=1,
                                                       add_self_loops=False)

            # item -> ft
            conv1[('item', f'to_feedback_{ft}', ft)] = GATConv(emb_dim, hidden_dim,
                                                       heads=heads,
                                                       add_self_loops=False)
            conv2[('item', f'to_feedback_{ft}', ft)] = GATConv(hidden_dim*heads, emb_dim,
                                                       heads=1,
                                                       add_self_loops=False)

        self.conv1 = HeteroConv(conv1, aggr='mean')
        self.conv2 = HeteroConv(conv2, aggr='mean')

        # LayerNorm и Dropout
        types = ['user', 'item'] + feedback_types
        self.norm1 = nn.ModuleDict({t: nn.LayerNorm(hidden_dim*heads) for t in types})
        self.norm2 = nn.ModuleDict({t: nn.LayerNorm(emb_dim) for t in types})
        self.dropout = nn.Dropout(dropout)

    def forward(self, data):
        x = {
            'user': self.user_emb(data['user'].node_id),
            'item': self.item_emb(data['item'].node_id)
        }
        for ft in self.feedback_types:
            x[ft] = self.fb_emb[ft](data[ft].node_id)

        h1 = self.conv1(x, data.edge_index_dict)
        # print(h1.keys())
        # Собираем позитивный и негативный вклад для user
        pos_msgs = torch.zeros_like(h1['user'])
        neg_msgs = torch.zeros_like(h1['user'])
        for ft in self.pos_types:
            pos_msgs += h1[ft]  
        for ft in self.neg_types:
            neg_msgs += h1[ft]

        # signed-aggregation: attraction от pos, repulsion от neg
        user_h1 = pos_msgs - neg_msgs
        user_h1 = F.leaky_relu(self.norm1['user'](user_h1))
        user_h1 = self.dropout(user_h1)

        h1['user'] = user_h1

        h2 = self.conv2(h1, data.edge_index_dict)
        # print(h2.keys())
        pos_msgs2, neg_msgs2 = 0, 0
        for ft in self.pos_types:
            pos_msgs2 += h2[ft]
        for ft in self.neg_types:
            neg_msgs2 += h2[ft]

        user_h2 = pos_msgs2 - neg_msgs2
        user_h2 = F.leaky_relu(self.norm2['user'](user_h2))

        return user_h2

In [21]:
data = prepare_hetero_data(train)
data

HeteroData(
  user={ node_id=[22363] },
  item={ node_id=[12095] },
  implicit_negative={ node_id=[22363] },
  expliсit_negative={ node_id=[22363] },
  explicit_positive={ node_id=[22363] },
  implicit_positive={ node_id=[22363] },
  (item, to_feedback_implicit_negative, implicit_negative)={ edge_index=[2, 17110] },
  (implicit_negative, feedback_to_item_implicit_negative, item)={ edge_index=[2, 17110] },
  (implicit_negative, to_user_implicit_negative, user)={ edge_index=[2, 22363] },
  (item, to_feedback_expliсit_negative, expliсit_negative)={ edge_index=[2, 17504] },
  (expliсit_negative, feedback_to_item_expliсit_negative, item)={ edge_index=[2, 17504] },
  (expliсit_negative, to_user_expliсit_negative, user)={ edge_index=[2, 22363] },
  (item, to_feedback_explicit_positive, explicit_positive)={ edge_index=[2, 90800] },
  (explicit_positive, feedback_to_item_explicit_positive, item)={ edge_index=[2, 90800] },
  (explicit_positive, to_user_explicit_positive, user)={ edge_index=[2, 2

In [22]:
train.item_id.nunique(), train.item_id.min(), train.item_id.max()

(12095, 0, 12094)

In [23]:
num_users = len(train['user_id'].unique())
num_items = train['item_id'].max() + 1
feedback_types = train['target'].unique().tolist()
model = SignedHeteroGNN(num_users, num_items, feedback_types)

In [24]:
model

SignedHeteroGNN(
  (user_emb): Embedding(22363, 32)
  (item_emb): Embedding(12095, 32)
  (fb_emb): ModuleDict(
    (implicit_negative): Embedding(22363, 32)
    (expliсit_negative): Embedding(22363, 32)
    (explicit_positive): Embedding(22363, 32)
    (implicit_positive): Embedding(22363, 32)
  )
  (conv1): HeteroConv(num_relations=12)
  (conv2): HeteroConv(num_relations=12)
  (norm1): ModuleDict(
    (user): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
    (item): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
    (implicit_negative): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
    (expliсit_negative): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
    (explicit_positive): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
    (implicit_positive): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
  )
  (norm2): ModuleDict(
    (user): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
    (item): LayerNorm((32,), eps=1e-05, elementwise_affine=True)

In [25]:
model(data)

tensor([[-8.0051e-03,  7.2226e-02,  3.2104e-01,  ...,  2.1603e+00,
         -3.7437e-03, -1.3732e-02],
        [-5.3090e-03,  3.6092e-02, -7.4186e-04,  ..., -1.6044e-02,
         -4.0745e-03, -1.2253e-02],
        [-6.9348e-03, -2.7053e-03, -5.1356e-03,  ..., -1.0003e-02,
          1.6777e+00, -4.0266e-03],
        ...,
        [-1.2594e-02,  3.9354e-01, -8.9218e-05,  ...,  7.2582e-01,
          7.1790e-01,  6.9499e-02],
        [ 5.3360e-01,  7.4546e-01, -1.3123e-02,  ..., -8.4632e-03,
         -8.0442e-03,  1.2919e+00],
        [ 9.5411e-01, -1.5633e-03, -4.5640e-03,  ...,  4.7633e-02,
         -2.1720e-03,  1.6018e-01]], grad_fn=<LeakyReluBackward0>)

In [26]:
test_df = test[['user_id', 'item_id']]
interactions = test_df.rename(columns={
    'user_id': Columns.User,
    'item_id': Columns.Item,
})

viewed_items = train.groupby("user_id")["item_id"].agg(set).to_dict()

In [27]:
def evaluate(model, train_data,
             test_batch_size, top_k,
             viewed_items, interactions,
             device, test_step):
    """
    Оцениваем модель по всем пользователям:
    - строим топ-K рекомендации
    - фильтруем уже просмотренные
    - считаем recall@K, precision@K, map@K
    """
    model.eval()
    model.to(device)
    num_users = train_data['user'].node_id.shape[0]
    test_top_k = top_k * 150

    item_emb = model.item_emb.weight
    item_emb_t = item_emb.t().detach()
    del item_emb
    gc.collect()

    all_scores = []
    with torch.no_grad():
        for i in range(0, num_users, test_batch_size):
            end = min(i + test_batch_size, num_users)
            batch_users = torch.arange(i, end).to(device)
            user_e = model(
                data=train_data.to(device)
            )
            rating = torch.mm(user_e[batch_users].detach(), item_emb_t)
            _, topk = torch.topk(rating, k=test_top_k, dim=1)
            all_scores.append(topk)

            del user_e, rating
            gc.collect()
    all_scores = torch.cat(all_scores, dim=0).cpu().numpy()

    users_list, items, ranks = [], [], []
    for u in range(num_users):
        seen = viewed_items.get(u, set())
        recs = all_scores[u]
        mask = ~np.isin(recs, list(seen))
        filtered = recs[mask][:top_k]
        for rank, it in enumerate(filtered, 1):
            users_list.append(u)
            items.append(int(it))
            ranks.append(rank)
    reco_df = pd.DataFrame({
        'user_id': users_list,
        'item_id': items,
        'rank': ranks
    })

    metrics = {
        f'map@{top_k}': MAP(k=top_k),
        f'precision@{top_k}': Precision(k=top_k),
        f'recall@{top_k}': Recall(k=top_k),
        f'ndcg@{top_k}': NDCG(k=top_k)
    }
    results = calc_metrics(metrics=metrics,
                           reco=reco_df,
                           interactions=interactions)
    print(f"Step {test_step} — Test metrics:")
    for name, val in results.items():
        print(f"  {name}: {val:.9f}")
        experiment.log_metric(f"Test {name} vs step", val, step=test_step)
    del all_scores
    gc.collect()

    model.to(device)
    train_data.to(device)
    model.train()
    return results

In [28]:
def train_model(model,
                train_data: HeteroData,
                edge_type: tuple,
                num_epochs: int = 10,
                lr: float = 1e-3,
                batch_size: int = 1024,
                device: str = None,
                print_every: int = 100,
                test_every: int = 500,
                top_k: int = 10,
                test_batch_size=2048):
    device = device or ('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    train_data = train_data.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    src, dst = train_data[edge_type].edge_index
    num_train = src.size(0)
    test_top_k = top_k * 300
    total_steps = 0
    print(f"Num of training examples: {num_train}")
    for epoch in range(1, num_epochs + 1):
        model.train()
        perm = torch.randperm(num_train, device=device)
        total_loss = 0.0
        step = 0
        for i in range(0, num_train, batch_size):
            idx = perm[i:i + batch_size]
            users = dst[idx]
            pos_items = src[idx]
            neg_items = torch.randint(0, model.item_emb.num_embeddings,
                                      size=pos_items.size(), device=device)
            optimizer.zero_grad()
            user_embs = model(train_data)[users]
            pos_emb = model.item_emb(pos_items)
            neg_emb = model.item_emb(neg_items)
            pos_score = (user_embs * pos_emb).sum(dim=1)
            neg_score = (user_embs * neg_emb).sum(dim=1)
            loss = -torch.log(torch.sigmoid(pos_score - neg_score) + 1e-15).mean()
            loss.backward()
            optimizer.step()
            total_loss += loss.item() * users.size(0)
            step += 1

            experiment.log_metric('Train BPR Loss vs step', loss.item(), step=total_steps)
            if step % print_every == 0 or step == 1:
                avg_loss = total_loss /  (step * batch_size)
                current_lr = optimizer.param_groups[0]['lr']
                d = (pos_score - neg_score).detach().cpu()
                print(f"Epoch {epoch}, Step {step}, LR: {current_lr:.6f}, Current Loss: {loss.item():.4f}, Avg Loss: {avg_loss:.4f}")
                print(f"Diff stats — min: {d.min():.4f}, max: {d.max():.4f}, mean: {d.mean():.4f}, std: {d.std():.4f}")
                print()

                experiment.log_metric('Diff stats (mean) vs step', d.mean(), step=total_steps)
                experiment.log_metric('Diff stats (std) vs step', d.std(), step=total_steps)

            del user_embs, pos_emb, neg_emb, pos_score, neg_score
            gc.collect()
            torch.cuda.empty_cache()
            
            if step % test_every == 0 or step == 1:
                evaluate(model, train_data,
                         test_batch_size, top_k,
                         viewed_items, interactions,
                         device, test_step=total_steps)
            total_steps += 1
        epoch_loss = total_loss / num_train
        experiment.log_metric(f'Train Loss vs epoch', epoch_loss, epoch=epoch - 1)
        print(f"Epoch {epoch} completed, Train Loss: {epoch_loss:.4f}")
    return model

In [29]:
experiment.log_parameters(hyperparameters)

In [30]:
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

In [31]:
edge_type = hyperparameters['train_edge_type']
num_epochs = hyperparameters['train_num_epochs']
lr = hyperparameters['train_lr']
batch_size = hyperparameters['train_batch_size']
print_every = hyperparameters['train_print_every']
test_every = hyperparameters['train_test_every']
top_k = hyperparameters['test_topk']
test_batch_size = hyperparameters['test_batch_size']
model = train_model(model,
                    data,
                    edge_type=edge_type,
                    num_epochs=num_epochs,
                    lr=lr,
                    batch_size=batch_size,
                    print_every=print_every,
                    test_every=test_every,
                    top_k=top_k,
                    test_batch_size=test_batch_size)

Num of training examples: 90800
Epoch 1, Step 1, LR: 0.000080, Current Loss: 2.3725, Avg Loss: 2.3725
Diff stats — min: -23.9808, max: 19.5933, mean: 0.0573, std: 5.7407

Step 0 — Test metrics:
  precision@10: 0.000589345
  recall@10: 0.000589345
  ndcg@10: 0.000492331
  map@10: 0.000115923
Epoch 1, Step 10, LR: 0.000080, Current Loss: 2.4116, Avg Loss: 2.3963
Diff stats — min: -20.4679, max: 20.4406, mean: -0.0146, std: 5.7534

Step 9 — Test metrics:
  precision@10: 0.000542197
  recall@10: 0.000542197
  ndcg@10: 0.000486465
  map@10: 0.000126167
Epoch 1, Step 20, LR: 0.000080, Current Loss: 2.5272, Avg Loss: 2.4080
Diff stats — min: -21.9587, max: 20.3354, mean: -0.2486, std: 5.7625

Step 19 — Test metrics:
  precision@10: 0.000518623
  recall@10: 0.000518623
  ndcg@10: 0.000468138
  map@10: 0.000122144
Epoch 1 completed, Train Loss: 2.4105
Epoch 2, Step 1, LR: 0.000080, Current Loss: 2.3555, Avg Loss: 2.3555
Diff stats — min: -18.3285, max: 18.3556, mean: -0.0400, std: 5.5807

Step 

In [32]:
torch.save(model, "gnn_model_mvl.model")
from IPython.display import FileLink

FileLink('gnn_model_mvl.model')

/kaggle/working/gnn_model_mvl.model

In [33]:
# del model
gc.collect()
torch.cuda.empty_cache()

In [34]:
log_model(
    experiment=experiment,
    model=model,
    model_name="GNN",
)

In [35]:
experiment.end()

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : diffGATGraphModel-beauty
COMET INFO:     url                   : https://www.comet.com/annanet/gnn-recommender/88062f9b711a4b7a8f06c94568841b6d
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     Diff stats (mean) vs step [300] : (-0.2486005425453186, 0.3308376669883728)
COMET INFO:     Diff stats (std) vs step [300]  : (1.1936794519424438, 5.762481689453125)
COMET INFO:     Test map@10 vs step [300]       : (0.0001159231569415445, 0.0001756901880664257)
COMET INFO:     Test ndcg@10 vs step [300]      : (0.00045594309913428487, 0.0006371242193328713)
COMET INFO:     Test precision@10 vs step [300] : (0.00042432814710042436, 0.000683639792550683